In [1]:
import sys
sys.path.append('/home/n31v/workspace/Fedot.Industrial')

import pandas as pd
from torchvision.models import resnet18
from fedot_ind.core.operation.optimization.svd_tools import load_svd_state_dict

from compression.metrics import *

In [2]:
df = pd.DataFrame(columns=['params, M', 'size, MB', 'GFLOP', 'inference time, ms', 'CPU time, ms', 'GPU time, ms'])

In [40]:
CIFAR_SIZE = (128, 3, 132, 132)
DEVICE = 'cpu'
SORT = f'self_{DEVICE}_time_total'
resnet18_for_cifar = resnet18(num_classes=10)
df.loc['Baseline ResNet18 CIFAR10'] = inference_metrics(model=resnet18_for_cifar, input_size=CIFAR_SIZE, device=DEVICE)
df

STAGE:2023-11-14 18:03:25 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:03:26 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:03:26 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:03:26 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:03:27 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:03:27 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:03:27 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:03:28 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:03:28 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:03:28 1787067:178706

,"params, M","size, MB",GFLOP,"inference time, ms","CPU time, ms","GPU time, ms"
Baseline ResNet18 CIFAR10,11.181642,44.765128,187.563442,421.4256,421.4256,0.0000
ResNet18 CIFAR10 one_layer,2.578952,10.354368,189.914631,14.0104,5.0893,8.9211
ResNet18 CIFAR10 two_layers,2.578952,10.354368,130.465980,18.8943,6.3264,12.5679
ResNet18 CIFAR10 three_layers,2.578952,10.354368,164.924475,21.7322,6.7803,14.9519


In [42]:
prof = profile(model=resnet18_for_cifar, input_size=CIFAR_SIZE, with_flops=False, device=DEVICE)
print(prof.key_averages().table(sort_by=SORT, row_limit=10))

STAGE:2023-11-14 18:04:23 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::mkldnn_convolution        66.67%     251.528ms        66.75%     251.855ms      12.593ms            20  
                  model_inference        13.75%      51.892ms       100.00%     377.285ms     377.285ms             1  
          aten::native_batch_norm         8.70%      32.827ms         8.73%      32.954ms       1.648ms            20  
                 aten::clamp_min_         3.73%      14.058ms         3.73%      14.058ms     826.941us            17  
    aten::max_pool2d_with_indices         3.64%      13.716ms         3.64%      13.716ms      13.716ms             1  
                       aten::add_       

STAGE:2023-11-14 18:04:23 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:04:23 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [43]:
resnet18_for_cifar_svd_one = resnet18(num_classes=10)
load_svd_state_dict(
    model=resnet18_for_cifar_svd_one,
    decomposing_mode='spatial',
    state_dict_path='/media/n31v/data/results/SVDRL/test/light_x3/model8458/Nov09_17-03/trained_model.sd.pt',
    forward_mode='one_layer',
)
df.loc['ResNet18 CIFAR10 one_layer'] = inference_metrics(model=resnet18_for_cifar_svd_one, input_size=CIFAR_SIZE, device=DEVICE)
df

STAGE:2023-11-14 18:04:44 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:04:44 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:04:44 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:04:44 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:04:45 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:04:45 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:04:45 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:04:46 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:04:46 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:04:46 1787067:178706

,"params, M","size, MB",GFLOP,"inference time, ms","CPU time, ms","GPU time, ms"
Baseline ResNet18 CIFAR10,11.181642,44.765128,187.563442,421.4256,421.4256,0.0000
ResNet18 CIFAR10 one_layer,2.578952,10.354368,189.914631,422.0418,422.0418,0.0000
ResNet18 CIFAR10 two_layers,2.578952,10.354368,130.465980,18.8943,6.3264,12.5679
ResNet18 CIFAR10 three_layers,2.578952,10.354368,164.924475,21.7322,6.7803,14.9519


In [47]:
prof = profile(model=resnet18_for_cifar_svd_one, input_size=CIFAR_SIZE, with_flops=False, device=DEVICE)
print(prof.key_averages().table(sort_by=SORT, row_limit=10))

STAGE:2023-11-14 18:05:18 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::mkldnn_convolution        61.33%     284.622ms        61.39%     284.923ms      14.246ms            20  
                  model_inference        18.16%      84.265ms       100.00%     464.085ms     464.085ms             1  
          aten::native_batch_norm         7.82%      36.305ms         7.85%      36.438ms       1.822ms            20  
    aten::max_pool2d_with_indices         3.06%      14.184ms         3.06%      14.184ms      14.184ms             1  
                 aten::clamp_min_         3.04%      14.122ms         3.04%      14.122ms     830.706us            17  
                       aten::add_       

STAGE:2023-11-14 18:05:19 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:19 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [48]:
resnet18_for_cifar_svd_two = resnet18(num_classes=10)
load_svd_state_dict(
    model=resnet18_for_cifar_svd_two,
    decomposing_mode='spatial',
    state_dict_path='/media/n31v/data/results/SVDRL/test/light_x3/model8458/Nov09_17-03/trained_model.sd.pt',
    forward_mode='two_layers',
)
df.loc['ResNet18 CIFAR10 two_layers'] = inference_metrics(model=resnet18_for_cifar_svd_two, input_size=CIFAR_SIZE, device=DEVICE)
df

STAGE:2023-11-14 18:05:29 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:05:29 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:29 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:05:30 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:05:30 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:30 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:05:30 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:05:31 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:31 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:05:31 1787067:178706

,"params, M","size, MB",GFLOP,"inference time, ms","CPU time, ms","GPU time, ms"
Baseline ResNet18 CIFAR10,11.181642,44.765128,187.563442,421.4256,421.4256,0.0000
ResNet18 CIFAR10 one_layer,2.578952,10.354368,189.914631,422.0418,422.0418,0.0000
ResNet18 CIFAR10 two_layers,2.578952,10.354368,130.465980,371.4678,371.4678,0.0000
ResNet18 CIFAR10 three_layers,2.578952,10.354368,164.924475,21.7322,6.7803,14.9519


In [49]:
prof = profile(model=resnet18_for_cifar_svd_two, input_size=CIFAR_SIZE, with_flops=False, device=DEVICE)
print(prof.key_averages().table(sort_by=SORT, row_limit=10))

STAGE:2023-11-14 18:05:42 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::mkldnn_convolution        55.15%     213.263ms        55.29%     213.800ms       5.345ms            40  
                  model_inference        16.35%      63.218ms       100.00%     386.676ms     386.676ms             1  
          aten::native_batch_norm        11.51%      44.505ms        11.57%      44.736ms       2.237ms            20  
    aten::max_pool2d_with_indices         3.99%      15.441ms         3.99%      15.441ms      15.441ms             1  
                 aten::clamp_min_         3.85%      14.885ms         3.85%      14.885ms     875.588us            17  
                       aten::add_       

STAGE:2023-11-14 18:05:43 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:43 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [50]:
resnet18_for_cifar_svd_three = resnet18(num_classes=10)
load_svd_state_dict(
    model=resnet18_for_cifar_svd_three,
    decomposing_mode='spatial',
    state_dict_path='/media/n31v/data/results/SVDRL/test/light_x3/model8458/Nov09_17-03/trained_model.sd.pt',
    forward_mode='three_layers',
)
df.loc['ResNet18 CIFAR10 three_layers'] = inference_metrics(model=resnet18_for_cifar_svd_three, input_size=CIFAR_SIZE, device=DEVICE)
df

STAGE:2023-11-14 18:05:50 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:05:51 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:51 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:05:51 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:05:52 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:52 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:05:52 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-11-14 18:05:53 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:05:53 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-11-14 18:05:53 1787067:178706

,"params, M","size, MB",GFLOP,"inference time, ms","CPU time, ms","GPU time, ms"
Baseline ResNet18 CIFAR10,11.181642,44.765128,187.563442,421.4256,421.4256,0.0
ResNet18 CIFAR10 one_layer,2.578952,10.354368,189.914631,422.0418,422.0418,0.0
ResNet18 CIFAR10 two_layers,2.578952,10.354368,130.465980,371.4678,371.4678,0.0
ResNet18 CIFAR10 three_layers,2.578952,10.354368,164.924475,483.0609,483.0609,0.0


In [51]:
prof = profile(model=resnet18_for_cifar_svd_three, input_size=CIFAR_SIZE, with_flops=False, device=DEVICE)
print(prof.key_averages().table(sort_by=SORT, row_limit=10))

STAGE:2023-11-14 18:06:32 1787067:1787067 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::mkldnn_convolution        60.73%     296.277ms        60.90%     297.071ms       4.951ms            60  
                  model_inference        18.86%      92.013ms       100.00%     487.808ms     487.808ms             1  
          aten::native_batch_norm         8.79%      42.877ms         8.84%      43.117ms       2.156ms            20  
                 aten::clamp_min_         3.28%      15.979ms         3.28%      15.979ms     939.941us            17  
                       aten::add_         2.79%      13.588ms         2.79%      13.588ms       1.698ms             8  
    aten::max_pool2d_with_indices       

STAGE:2023-11-14 18:06:33 1787067:1787067 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-11-14 18:06:33 1787067:1787067 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
